# Chap05 - 오차역전파법 Backpropagation

> [Chap04 - 신경망 학습](https://github.com/ExcelsiorCJH/DLFromScratch/blob/master/Chap04-Neural_Network_Traing/Chap04-Neural_Network_Training.ipynb)에서는 가중치 매개변수의 기울기를 미분을 이용해 구했다. 이러한 방법은 간단하지만 시간이 오래 걸리는 단점이 있다. 
이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 **오차역전파법(backpropagation)**에 대해 알아보도록 하자.

## 5.1 계산 그래프

**계산 그래프(computational graph)**는 계산 과정을 그래프로 나타낸 것이며, **노드**(node)와 **엣지**(edge)로 표현된다. 노드는 연산을 정의하며, 엣지는 데이터가 흘러가는 방향을 나타낸다.

### 5.1.1 계산 그래프로 풀다

> 현빈 군은 슈퍼에서 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원입니다. 소비세가 10%일 때 지불 금액을 구하라.

<img src="./images/5-3.png" width="75%" height="75%"/>

1. 계산 그래프를 구성한다.
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다. → **순전파**(forward propagation)

### 5.1.2 국소적 계산

계산 그래프의 특징은 '국소적 계산'을 통해 최종 결과를 얻는 것이다. 즉, '자신과 직접 관계된' 범위 내에서만 계산이 이루어 진다. 

<img src="./images/5-4.png" width="75%" height="75%"/>

### 5.1.3 왜 계산 그래프로 푸는가?

계산그래프의 장점은 다음과 같다.

- **국소적 계산**을 통해 각 노드의 계산에 집중하여 문제를 단순화할 수 있다.
- **역전파**를 통해 '미분'을 효율적으로 계산할 수 있다.

'사과 가격이 오르면 최종 금액에 어떠한 영향을 주는가'에 대해서 **사과 가격에 대한 지불 금액의 미분**을 구해 계산할 수 있다. 사과의 값을 $x$, 지불 금액을 $L$라 했을 때, $\frac{\partial L}{\partial x}$를 구하는 것이다. 이러한 미분 값은 사과 값($x$)가 '아주 조금'올랐을 때 지불 금액($L$)이 얼마나 증가하는지를 나타낸다.

<img src="./images/5-5.PNG" width="75%" height="75%"/>

## 5.2 연쇄 법칙 - Chain Rule

### 5.2.1 계산 그래프의 역전파

먼저, 역전파 계산 예제로 $y=f(x)$ 의 역전파를 계산해보자. 

<img src="./images/5-6.PNG" width="50%" height="50%"/>

위의 그림에서 처럼 역전파 계산 순서는 신호 $E$에 노드($f$)의 국소적 미분 $\left( \frac{\partial y}{\partial x} \right)$을 곱한 후 엣지(edge)를 통해 다음 노드로 전달하는 것이다. 여기서 국소적 미분은 순전파 때의 $y = f(x)$에 대한 미분을 구하는 것이고, 이것은 $x$에 대한 $y$의 미분 $\left( \frac{\partial y}{\partial x} \right)$을 구한다는 의미이다.

### 5.2.2 연쇄법칙이란?

> *합성함수의 미분은 합성 함수를 구헝하는 각 함수의 미분의 곱으로 나타낼 수 있다.*

$$
t = x + y
$$

$$
z = t^2
$$

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
$$

$$
\frac{\partial z}{\partial t} = 2t
$$

$$
\frac{\partial t}{\partial x} = 1
$$

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t \cdot 1 = 2(x + y)
$$

### 5.2.3 연쇄법칙과 계산 그래프

5.2.2의 예제 식을 계산 그래프로 나타내면 다음과 같다.

<img src="./images/5-8.png" width="90%" height="90%"/>

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

$$
z = x + y
$$

$$
\frac{\partial z}{\partial x} = 1
$$

$$
\frac{\partial z}{\partial y} = 1
$$

덧셈 노드의 역전파는 입력값을 그대로 흘려보낸다.

<img src="./images/5-9.PNG" width="80%" height="80%"/>

위의 $z=x+y$계산은 큰 계산 그래프의 중간 어딘가에 존재한다고 가정했기 때문에, 이 계산 그래프의 앞부분(상류)에서부터 $\frac{\partial L}{\partial z}$가 전해졌다고 가정한다.

<img src="./images/5-10.png" width="70%" height="70%"/>

### 5.3.2 곱셈 노드의 역전파

$$
z =xy
$$

$$
\frac{\partial z}{\partial x} = y 
$$

$$
\frac{\partial z}{\partial y} = x
$$

<img src="./images/5-12.PNG" width="80%" height="80%"/>

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

`forward()`는 순전파, `backward()`는 역전파이다.

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

<img src="./images/5-16.PNG" width="75%" height="75%"/>

In [14]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print('%d' % price)

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("%.1f, %d, %d" % (dapple, dapple_num, dtax))

220
2.2, 110, 200


### 5.4.2 덧셈 계층

In [17]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

<img src="./images/5-17.PNG" width="75%" height="75%"/>

In [21]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print('%d' % price)
print("%d, %.1f, %.1f, %d, %d" % (dapple_num, dapple, dorange, dorange_num, dtax))

715
110, 2.2, 3.3, 165, 650


## 5.5 활성화 함수 계층 구현하기